In [1]:
import os
import time
import pprint

import tiktoken
from langchain.text_splitter import RecursiveCharacterTextSplitter

tokenizer = tiktoken.get_encoding("cl100k_base")

def tiktoken_len(text):
    tokens = tokenizer.encode(text)
    return len(tokens)

from langchain.embeddings import HuggingFaceEmbeddings

model_name = 'jhgan/ko-sbert-nli'
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True}
ko = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

d:\ProgramData\anaconda\envs\llm\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
loader=DirectoryLoader('assets/doc/secretgarden_script',
                       glob="*.pdf",
                       loader_cls=PyPDFLoader)

pages=loader.load_and_split()

print(pages)

[Document(page_content='시크릿가든 01 1시크릿 가든 1부\nS#1. 삭제\nS#2. 시크릿 가든 곳곳 / 몽타주 . 낮.\n울창한 숲과, 중세 유럽의 대 장원(莊園) 같은 대저택 구석구석 보여진다 .  \n* 가을 단풍잎 사이로 보이는 청명하고 푸른 하늘…\n* 잔디 언덕과 벤치… 오브제 스테이지 … 온갖 나무들과 꽃들… \n* 아름다운 도서관과 흙집돌집 … \n건물 벽을 타고 올라가는 담쟁이 넝쿨… \n* 숲과 나무로 둘러싸인 수영장과 다섯 개의 연못과 연못에 비친 나무 그림자들 …\n* 눈을 의심케 하는 돔 형태의 거대한 공연장 … \n* 그리팅 하우스의 촛불로 꾸며진 멋진 야경…\n기상캐스터  (NA) 여러분 혹시 ‘인디안 썸머’를 아십니까 ? 추운 겨울이 오기 전, 가을 끝에 \n   찾아오는 잠깐의 여름을 뜻하는데요 . 인디언들은 다시 찾아온 그 짧은 여름 동안, \n   겨울을 나기 위한 사냥을 했다고 합니다 . \nS#3. 시크릿 가든 / 주원의 수상가옥 / 다리 + 거실. 낮.\n수상 식물들 예쁘게 피어 있는 연못… \n연못가에 지어진 멋진 수상가옥 . \n연못 위에 걸쳐진 다리 건너 카메라 실내로 들어간다 . \n기상캐스터  (NA) 그래서 인디안 썸머를 신이 내린 선물이라고 불렀다고 하죠.\n고급스러워 보이는 심플한 실내. \nTV 화면에서 일기예보 방송되고 있다.  \n기상캐스터 올 가을 찾아 올 인디안 썸머는 전례 없이 많은 비 소식을 동반하고 있는데요 . \n주원    (말끔한 차림으로 2층 계단 턱턱턱 내려오는 ) \n기상캐스터  (E) 이 비를 맞으면 왠지 특별한 신의 선물을 받을 수 있는 건 아닐까요 ? \n주원    (리모컨 집어 들고 끄려는데 )\n기상캐스터  하지만 신의 선물일지 장난일지는 비를 맞아봐야 알 수 있겠죠 , 김주원 씨?\n   (주원 향해 생긋…) \n주원    (헉! 바, 방금 저 여자 뭐란 거야? 김주원 ? 나?)', metadata={'source': 'assets\\doc\\secre

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=50, chunk_overlap=5, length_function=tiktoken_len)
docs = text_splitter.split_documents(pages)

print('[+] Split Done!!')

[+] Split Done!!


In [5]:
# save to disk
from langchain.vectorstores import Chroma

saved_db = Chroma.from_documents(docs, ko, persist_directory='chroma_db')
loaded_db = Chroma(persist_directory="chroma_db", embedding_function=ko)

print('[+] Embedding done!!')

[+] Embedding done!!


In [6]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.llms import LlamaCpp

model_path = "assets/models/llama-2-7b-chat.Q4_K_M.gguf"

llama = LlamaCpp(
    model_path=model_path,
    streaming=True,
)

print('[+] Ready to LLM!!')

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from assets/models/llama-2-7b-chat.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32  

[+] Ready to LLM!!


AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | MATMUL_INT8 = 0 | 
Model metadata: {'general.name': 'LLaMA v2', 'general.architecture': 'llama', 'llama.context_length': '4096', 'llama.rope.dimension_count': '128', 'llama.embedding_length': '4096', 'llama.block_count': '32', 'llama.feed_forward_length': '11008', 'llama.attention.head_count': '32', 'tokenizer.ggml.eos_token_id': '2', 'general.file_type': '15', 'llama.attention.head_count_kv': '32', 'llama.attention.layer_norm_rms_epsilon': '0.000001', 'tokenizer.ggml.model': 'llama', 'general.quantization_version': '2', 'tokenizer.ggml.bos_token_id': '1', 'tokenizer.ggml.unknown_token_id': '0'}


In [7]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(llm=llama,
                                 chain_type="stuff",
                                 retriever=loaded_db.as_retriever(
                                     search_type="mmr",
                                     search_kwargs={'k':3, 'fetch_k': 10}
                                 ),
                                 return_source_documents=True
                                 )

print('[+] Ready to chain\n')

[+] Ready to chain



In [8]:
query = "주원은 죽니?"

start_time = time.time()
print(time.ctime(start_time))
result = qa(query)
print("\nElapsed Time:", time.ctime(time.time()-start_time))

pprint.pprint(result)

Sat Mar  2 16:33:54 2024


d:\ProgramData\anaconda\envs\llm\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(

llama_print_timings:        load time =    3869.49 ms
llama_print_timings:      sample time =      43.33 ms /   101 runs   (    0.43 ms per token,  2330.73 tokens per second)
llama_print_timings: prompt eval time =   37095.58 ms /   114 tokens (  325.40 ms per token,     3.07 tokens per second)
llama_print_timings:        eval time =   35357.31 ms /   100 runs   (  353.57 ms per token,     2.83 tokens per second)
llama_print_timings:       total time =   73176.44 ms /   214 tokens



Elapsed Time: Thu Jan  1 09:01:15 1970
{'query': '주원은 죽니?',
 'result': ' No, 주원 is not dead.\n'
           '\n'
           'Context:\n'
           '\n'
           '* 주원 (Joo-won) is a Korean actor and singer.\n'
           '* He was born on March 21, 1981 in Seoul, South Korea.\n'
           '* He is best known for his roles in dramas such as "Sungkyunkwan '
           'Scandal" and "Moon Embracing the Sun".\n'
           '* He has also released several singles as a singer.',
 'source_documents': [Document(page_content='주원 뭐요?', metadata={'page': 18, 'source': 'assets\\doc\\secretgarden_script\\시크릿가든+01부.hwp.pdf'}),
                      Document(page_content='주원 뭐?', metadata={'page': 1, 'source': 'assets\\doc\\secretgarden_script\\시크릿가든+19부.hwp.pdf'}),
                      Document(page_content='주원 (뭐래는 거야? 돌아보면 ! 헐!)', metadata={'page': 4, 'source': 'assets\\doc\\secretgarden_script\\시크릿가든+05부.hwp.pdf'})]}
